In [1]:
import pandas as pd
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)


In [2]:
data = pd.read_csv("../dataset/streamspot/processed.csv")
log_df = pd.DataFrame(data)
display(log_df)

,5,process,6,file,execve,1,benign
0,5,process,7,file,execve,1,benign
1,5,process,8,file,execve,1,benign
2,5,process,9,file,execve,1,benign
3,5,process,10,file,access,1,benign
4,5,process,1,MAP_ANONYMOUS,mmap2,1,benign
...,...,...,...,...,...,...,...
89770799,5035516,process,5037797,file,close,600,benign
89770800,5035516,process,5042521,file,unlink,600,benign
89770801,5035516,process,5037797,file,close,600,benign
89770802,5035516,process,5044046,file,unlink,600,benign


In [3]:
def build_extended_graph(df):
    """
    Xây dựng đồ thị nguồn gốc mở rộng từ DataFrame log.
    Mỗi cạnh (hành vi) sẽ được biểu diễn bằng một nút ảo.
    """
    G_extended = nx.Graph() # Sử dụng đồ thị vô hướng cho GCN đơn giản

    for index, row in tqdm(df.iterrows()):
        src_node, dst_node = row['src'], row['dst']
        edge_type = row['edge_type']

        # Tạo nút ảo cho hành vi (system call)
        virtual_node_id = f"event_{index}"

        # 1. Thêm các nút thực thể nếu chưa có
        if not G_extended.has_node(src_node):
            node_type = 'socket' if 'sock_' in src_node else 'process' if 'proc_' in src_node else 'file'
            G_extended.add_node(src_node, type=node_type, name=src_node)
        if not G_extended.has_node(dst_node):
            node_type = 'socket' if 'sock_' in dst_node else 'process' if 'proc_' in dst_node else 'file'
            G_extended.add_node(dst_node, type=node_type, name=dst_node)

        # 2. Thêm nút ảo và gán thuộc tính là loại hành vi
        G_extended.add_node(virtual_node_id, type='event', name=edge_type)

        # 3. Nối nút nguồn và đích vào nút ảo
        G_extended.add_edge(src_node, virtual_node_id)
        G_extended.add_edge(virtual_node_id, dst_node)

    return G_extended

# Xây dựng đồ thị
provenance_graph = build_extended_graph(log_df)

print(f"Đồ thị nguồn gốc mở rộng đã được tạo với {provenance_graph.number_of_nodes()} nút và {provenance_graph.number_of_edges()} cạnh.")

# Trực quan hóa đồ thị
pos = nx.spring_layout(provenance_graph, k=0.8)
node_colors = []
for node in provenance_graph.nodes(data=True):
    if node[1]['type'] == 'process':
        node_colors.append('skyblue')
    elif node[1]['type'] == 'file':
        node_colors.append('lightgreen')
    elif node[1]['type'] == 'socket':
        node_colors.append('salmon')
    else: # Nút ảo (event)
        node_colors.append('grey')

nx.draw(provenance_graph, pos, with_labels=True, node_color=node_colors, font_size=8)
plt.title("Đồ thị Nguồn gốc Mở rộng")
plt.show()

0it [00:30, ?it/s]


KeyError: 'src'

In [ ]:
def mock_llm_annotator(node_id, attributes):
    """
    Hàm giả lập LLM để gán nhãn cho nút.
    Trong thực tế, hàm này sẽ gọi đến một LLM thật với RAG.
    Logic giả lập:
    - Độc hại (malicious): Nếu truy cập file nhạy cảm, kết nối tới server đã biết là C2.
    - Đáng ngờ (suspicious): Ghi vào thư mục /tmp, sử dụng các tiến trình mạng cơ bản.
    - Lành tính (benign): Các trường hợp còn lại.
    """
    # Định nghĩa các lớp: 0: lành tính, 1: đáng ngờ, 2: độc hại

    # Logic cho nút thực thể
    if attributes['type'] == 'file':
        if '/etc/passwd' in node_id or '/root/.ssh' in node_id:
            return {'label': 2, 'confidence': 95, 'reason': 'Truy cập file hệ thống cực kỳ nhạy cảm.'}
        if '/tmp/' in node_id:
            return {'label': 1, 'confidence': 70, 'reason': 'Ghi vào thư mục tạm, thường dùng để tải payload.'}

    if attributes['type'] == 'socket':
        if 'c2_server' in node_id or 'exfil_server' in node_id:
            return {'label': 2, 'confidence': 98, 'reason': 'Kết nối tới máy chủ C2 hoặc máy chủ rút ruột dữ liệu.'}

    # Logic cho nút ảo (hành vi)
    if attributes['type'] == 'event':
        if attributes['name'] == 'EXECVE':
             return {'label': 1, 'confidence': 75, 'reason': 'Thực thi một tiến trình mới, cần theo dõi.'}

    # Mặc định là lành tính
    return {'label': 0, 'confidence': 80, 'reason': 'Hành vi thông thường.'}

# Áp dụng LLM Annotator và chọn lọc nút
node_labels = {}
initial_labels = {} # Dict chứa các nhãn ban đầu cho GNN

# Duyệt qua các nút, chỉ chọn những nút có độ tin cậy cao và không phải lành tính
print("--- Bắt đầu quá trình gán nhãn giả lập bằng LLM ---")
for node_id, attrs in provenance_graph.nodes(data=True):
    annotation = mock_llm_annotator(node_id, attrs)

    # Confidence-aware Node Selection:
    # Chỉ chọn các nút được gán nhãn "đáng ngờ" hoặc "độc hại" với độ tin cậy > 80%
    if annotation['label'] > 0 and annotation['confidence'] > 80:
        initial_labels[node_id] = annotation['label']
        print(f"  - Nút '{node_id}' được chọn làm hạt giống huấn luyện với nhãn '{annotation['label']}' (Lý do: {annotation['reason']})")

print(f"\n=> Đã chọn được {len(initial_labels)} nút làm dữ liệu huấn luyện ban đầu.")

In [ ]:
# Tạo mapping từ tên nút sang chỉ số (integer)
node_list = list(provenance_graph.nodes())
node_mapping = {node: i for i, node in enumerate(node_list)}

# Tạo ma trận đặc trưng (node features) - one-hot encoding theo loại nút
node_types = ['process', 'file', 'socket', 'event']
num_nodes = provenance_graph.number_of_nodes()
num_features = len(node_types)
x = torch.zeros((num_nodes, num_features))

for i, node_id in enumerate(node_list):
    node_type = provenance_graph.nodes[node_id]['type']
    type_idx = node_types.index(node_type)
    x[i, type_idx] = 1

# Chuyển đổi đồ thị networkx sang PyG
pyg_graph = from_networkx(provenance_graph)
pyg_graph.x = x

# Tạo nhãn (y) và train_mask
y = torch.full((num_nodes,), -1, dtype=torch.long) # -1 cho các nút không có nhãn
train_mask = torch.zeros(num_nodes, dtype=torch.bool)

for node_id, label in initial_labels.items():
    node_idx = node_mapping[node_id]
    y[node_idx] = label
    train_mask[node_idx] = True

pyg_graph.y = y
pyg_graph.train_mask = train_mask

print("\nĐối tượng dữ liệu PyTorch Geometric:")
print(pyg_graph)
print(f"\nSố nút có nhãn để huấn luyện: {pyg_graph.train_mask.sum().item()}")

In [ ]:
class GCN_APT_Detector(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Lớp GCN thứ nhất
        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        # Lớp GCN thứ hai
        x = self.conv2(x, edge_index)

        return torch.log_softmax(x, dim=1)

# Khởi tạo mô hình
# Số lớp = 3 (lành tính, đáng ngờ, độc hại)
model = GCN_APT_Detector(num_node_features=num_features, hidden_channels=16, num_classes=3)
print("Mô hình GNN đã được khởi tạo:")
print(model)

In [ ]:
# Cài đặt cho việc huấn luyện
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(pyg_graph)
    # Chỉ tính loss trên các nút có nhãn trong train_mask
    loss = criterion(out[pyg_graph.train_mask], pyg_graph.y[pyg_graph.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

print("Bắt đầu quá trình huấn luyện mô hình GNN...")
losses = []
for epoch in range(1, 201):
    loss = train()
    losses.append(loss)
    if epoch % 20 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

# Trực quan hóa quá trình học
plt.figure(figsize=(8, 5))
plt.plot(losses)
plt.title("Loss trong quá trình huấn luyện")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

In [ ]:
# Đưa mô hình về chế độ đánh giá
model.eval()

# Lấy dự đoán cho tất cả các nút
with torch.no_grad():
    out = model(pyg_graph)
    _, pred = out.max(dim=1)

print("--- Kết quả Phát hiện Tấn công ---")
print("Nhãn dự đoán cho tất cả các nút:", pred.numpy())

# In ra các nút được phát hiện là đáng ngờ hoặc độc hại
# (loại trừ các nút đã được gán nhãn ban đầu)
detected_suspicious = []
detected_malicious = []

for i, label in enumerate(pred):
    # Nếu nút này không có trong tập huấn luyện ban đầu
    if not pyg_graph.train_mask[i]:
        if label == 1: # Đáng ngờ
            detected_suspicious.append(node_list[i])
        elif label == 2: # Độc hại
            detected_malicious.append(node_list[i])

print("\n[+] Các nút mới được phát hiện là ĐỘC HẠI (Malicious):")
if detected_malicious:
    for node in detected_malicious:
        print(f"  - {node}")
else:
    print("  (Không có)")

print("\n[+] Các nút mới được phát hiện là ĐÁNG NGỜ (Suspicious):")
if detected_suspicious:
    for node in detected_suspicious:
        print(f"  - {node}")
else:
    print("  (Không có)")